In [1]:
#imports
import pandas as pd
import numpy as np
import os
import warnings
from datetime import datetime as dt
from datetime import timedelta
import timeit

# configurations
warnings.filterwarnings('ignore')
pd.options.display.max_colwidth = None
pd.options.display.max_columns = None

In [2]:
# Helper Functions
def combine_tweets_user(df):
    """
    used for combine_tweets function
    
    Arguments:
    df -- A dataframe of a single user's tweets and created_at dates. 
    
    Returns:
    combined_df -- A dataframe of grouped user tweets as lists before the next open/close datetime (see combine_tweets function)
    """
    # convert dates to datetime and split
    datetimes = pd.to_datetime(df['created_at'])
    df['time'], df['date'] = datetimes.dt.time, datetimes.dt.date
    
    # get open and closing times
    open_t = dt.strptime('09:30:00', '%H:%M:%S').time()
    close_t = dt.strptime('16:00:00', '%H:%M:%S').time()
    
    # filter dataframes based off when tweet occurs
    before_df = df[df['time'] < open_t]
    during_df = df[((df['time'] >= open_t) & (df['time'] < close_t))]
    after_df = df[df['time'] >= close_t]
    
    # reset tweet times to next open or close time
    before_df['time'] = open_t
    during_df['time'] = close_t
    
    # if tweet occurs after close, set it to next day and set time to open
    after_df['date'] += timedelta(days=1)
    after_df['time'] = open_t

    # reassemble dataframes with updated dates/times
    df = before_df.append(during_df).append(after_df)
    
    # sort by date and time
    df['date'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
    df = df.sort_values(by='date', ascending=True)
    
    tweet = []
    combined_df = pd.DataFrame(index=sorted(list(set(df['date']))))
    combined_df['tweet'] = [[] for _ in range(len(combined_df))]
    
    # iterate over dataframe and add tokenized tweets to matching dates
    #tokenizer = TweetTokenizer()
    for i in range(len(combined_df)):
        time = combined_df.index[i]
        matching_dates = df[df['date'] == time]
        for j in range(len(matching_dates)):
            combined_df['tweet'].iloc[i].append(matching_dates['tweet'].iloc[j] + " " + matching_dates['created_at'].iloc[j])
            #combined_df.iloc[i]['tweet'].append(df.iloc[i]['tweet'])
    # add number of tweets and username and return
    combined_df['num_tweets'] = combined_df['tweet'].apply(lambda x: len(x) )
    combined_df['username'] = df['username'].unique()[0]
    return combined_df

def combine_tweets(full_df, verbose=False):
    """
    group tweets together by next open/close date
    
    Arguments:
    full_df -- A dataframe of users tweets and created_at dates. 
    
    Returns:
    merged_df -- A dataframe of grouped user tweets as lists before the next open/close datetime
  
    A tweet is grouped via the following criteria:
    market_open = 09:30:00 EST
    market_close = 16:00:00 EST
    
    - If tweet created_at occurs before market_open on same date, it is grouped as before_market_tweets. 
    - It tweet is on or after market_open but before market_close, (i.e. during market hours) it is grouped as during_market_tweets. 
    - If tweet is on or after market_close, it is grouped as after_market_tweets.
    """
    
    users = full_df.username.unique()
    merged_df = pd.DataFrame()
    if verbose:
        max_length = 11963
        start = timeit.default_timer()
        print('')
        print('digesting dataframes....')
        print('---{:.2f} % complete -------'.format(0.0))
    for user in users:
        merged_df = merged_df.append(combine_tweets_user(full_df[full_df['username'] == user]))
        if verbose:
            print('')
            print('--{:.2f} % complete -------'.format(100*len(merged_df) / max_length))
    if verbose:
        stop = timeit.default_timer()
        print('digestion completed ---- runtime {:.2f} seconds'.format(stop - start))
    return merged_df.reset_index(drop=False).rename(columns={'index':'date'})
    
def combine_tweets_stocks(ceos_merged, stocks_full):
    """
    merges combined user tweets with repective stock data
    
    Arguments:
    ceos_merged -- output of the combine_tweets function
    stocks_full -- a collection of stock data with the ticker labels as a column
    
    Returns:
    new_df -- the resultant dataframe following merging the stocks and the tweets dataframes 
    """
    new_df = pd.DataFrame()
    for user in ceos_merged['username'].unique():
        ticker = handles_tickers[user]
        tweet_df = ceos_merged[ceos_merged['username'] == user]
        stock_df = stocks_full[stocks_full['ticker'] == ticker]
        new_df = new_df.append(tweet_df.merge(stock_df, how='left', on='date')).dropna(subset=['ticker'])  
    return new_df

def fix_closed_market_days(combined_df):
    """
    Collects tweets that fall on days the market is closed. Updates dataframe and returns combined tweets
    
    Arguments:
    combined_df -- the output of combine_tweets_stocks where there are null values in the returns column
    
    Returns:
    new_df -- updated dataframe where there are no null values and the tweets that fell on null values are added to the next non-null return date
    """
    new_df = pd.DataFrame()
    for user in combined_df['username'].unique():
        user_df = combined_df[combined_df['username'] == user].set_index('date')
        user_return_null = user_df[user_df['return'].isnull()]
        user_price_not_null = user_df[user_df['price'].notnull()]
        user_return_not_null = user_price_not_null[user_price_not_null['return'].notnull()]
        saved_time_point = user_df.index.min() - timedelta(days=1)
        while saved_time_point <= user_return_null.index.max():
            combined_tweet = []
            first_time_point = user_return_null[user_return_null.index > saved_time_point].iloc[0].name
            next_time_point = user_return_not_null[user_return_not_null.index > first_time_point].iloc[0].name
            temp_df = user_df.loc[first_time_point : next_time_point]
            saved_time_point = temp_df.index.max()
            for j in range(len(temp_df)):
                            combined_tweet += temp_df['tweet'].iloc[j]
            user_return_not_null.loc[next_time_point, 'tweet'] = np.nan
            user_return_not_null.at[next_time_point, 'tweet'] = combined_tweet
            user_return_not_null.at[next_time_point, 'num_tweets'] = int(len(user_return_not_null.loc[next_time_point, 'tweet']))
        new_df = new_df.append(user_return_not_null)
    return new_df

def organize_stocks(df): 
    """
    Combines the open/close dates and prices into a single dataframe
    """
    stock_open = pd.DataFrame()
    stock_close = pd.DataFrame()

    stock_open['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d 09:30:00'))
    stock_close['date'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d 16:00:00'))

    stock_open['price'] = df['open']
    stock_close['price'] = df['close']
    
    start_date_open = dt.strftime(stock_open.date.min(), '%Y-%m-%d %H:%M:%S')
    start_date_close = dt.strftime(stock_close.date.min(), '%Y-%m-%d %H:%M:%S')

    end_date_open = dt.strftime(stock_open.date.max(), '%Y-%m-%d %H:%M:%S')
    end_date_close = dt.strftime(stock_close.date.max(), '%Y-%m-%d %H:%M:%S')
    
    date_indx_open = pd.Series(pd.date_range(start_date_open, end_date_open), name='date')
    date_indx_close = pd.Series(pd.date_range(start_date_close, end_date_close), name='date')
    
    stock_open = pd.merge(date_indx_open, stock_open, how='left')
    stock_close = pd.merge(date_indx_close, stock_close, how='left')
    
    stock = pd.concat([stock_open, stock_close])
    return stock.sort_values(by='date', ascending=True)

# Load the data

In [3]:
tweets = pd.read_pickle('./data/ceos.pkl')
stock_names = ['TSLA', 'BOX', 'TWTR', 'CRM', 'SPCE', 'TMUS']
stocks = {stock:pd.read_pickle(f'data/{stock}.pkl') for stock in stock_names}
tweets.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video',
       'thumbnail', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest', 'search'],
      dtype='object')

In [4]:
tweets[['created_at', 'username','tweet']].head()

,created_at,username,tweet
0,2021-07-20 07:55:22 EDT,elonmusk,@vincent13031925 Great to hear!
1,2021-07-20 03:46:56 EDT,elonmusk,@blueorigin Best of luck tomorrow!
2,2021-07-19 21:22:49 EDT,elonmusk,@SpacePadreIsle @SpaceX Haha
3,2021-07-19 20:17:03 EDT,elonmusk,"@TLPN_Official @SpaceX Depending on progress with Booster 4, we might try a 9 engine firing on Booster 3"
4,2021-07-19 20:07:03 EDT,elonmusk,Full test duration firing of 3 Raptors on Super Heavy Booster!


In [5]:
stocks['TSLA'].head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2016-08-23,44.863998,45.698002,44.560001,44.967999,23922000,0,0.0
1,2016-08-24,45.410000,45.430000,44.444000,44.523998,12853500,0,0.0
2,2016-08-25,44.622002,44.759998,44.153999,44.192001,8812500,0,0.0
3,2016-08-26,44.428001,44.571999,43.764000,43.998001,11195000,0,0.0
4,2016-08-29,44.029999,44.080002,43.000000,43.040001,16331500,0,0.0


In [6]:
# group the tweets together
tweets_combined = combine_tweets(tweets, verbose=True)
tweets_combined.head()


digesting dataframes....
---0.00 % complete -------

--18.38 % complete -------

--26.44 % complete -------

--38.93 % complete -------

--54.63 % complete -------

--71.44 % complete -------

--100.00 % complete -------
digestion completed ---- runtime 54.73 seconds


,date,tweet,num_tweets,username
0,2016-07-20 16:00:00,"[Finishing off the plan while listening to the soundtrack from Gatsby. Seems appropriate... 2016-07-20 10:37:26 EDT, Not easy to convey irony in a tweet 2016-07-20 11:21:56 EDT, Except for the tweet about large amounts of crack (actually just small amounts) 2016-07-20 12:56:28 EDT, @AnnaDotBlue busted 2016-07-20 14:01:55 EDT, @vitiral yup 2016-07-20 14:25:26 EDT, Post should go live on Tesla website around 5pm 2016-07-20 15:07:36 EDT, California time 2016-07-20 15:07:53 EDT, @princeghosh97 Lana 2016-07-20 10:58:31 EDT]",8,elonmusk
1,2016-07-21 09:30:00,"[Tesla Master Plan, Part Deux https://t.co/XQeTlsoEdE 2016-07-20 20:57:51 EDT, @danahull It's inspired by some of the California Custom VW combi design art 2016-07-21 00:54:09 EDT, @danahull Jerome is driving Tesla Semi &amp; doing a great job with his team. At Daimler, he led their most successful semi truck program ever. 2016-07-21 00:58:40 EDT, Autopilot prevents serious injury or death of a pedestrian in NY (owner anecdote confirmed by vehicle logs) https://t.co/NceuqckqCK 2016-07-21 01:53:01 EDT, @elonmusk now 2Pac 2016-07-20 16:20:36 EDT, @katiefehren Both should have been done under the same corporate umbrella from the beginning 2016-07-20 21:06:47 EDT, @MRamseyWSJ Great game. Whole Fallout series is amazing. 2016-07-20 21:39:21 EDT]",7,elonmusk
2,2016-07-22 09:30:00,[Looking forward to the Gigafactory opening party on Friday next week. The scale blows me away every time I see it. 2016-07-22 01:50:03 EDT],1,elonmusk
3,2016-07-23 09:30:00,"[ https://t.co/Kgkz1Z3FwJ 2016-07-22 21:34:30 EDT, @vicentes as long as I'm alive 2016-07-22 21:35:46 EDT]",2,elonmusk
4,2016-07-25 16:00:00,[@ID_AA_Carmack Maybe as a future optimization. Would only matter if it was extremely big. 2016-07-25 11:37:39 EDT],1,elonmusk


In [7]:
tweets_combined['tweet'].iloc[0]

['Finishing off the plan while listening to the soundtrack from Gatsby. Seems appropriate... 2016-07-20 10:37:26 EDT',
 'Not easy to convey irony in a tweet 2016-07-20 11:21:56 EDT',
 'Except for the tweet about large amounts of crack (actually just small amounts) 2016-07-20 12:56:28 EDT',
 '@AnnaDotBlue busted 2016-07-20 14:01:55 EDT',
 '@vitiral yup 2016-07-20 14:25:26 EDT',
 'Post should go live on Tesla website around 5pm 2016-07-20 15:07:36 EDT',
 'California time 2016-07-20 15:07:53 EDT',
 '@princeghosh97 Lana 2016-07-20 10:58:31 EDT']

In [8]:
# create a mapping between twitter handles and company stock tickers
handles = tweets['username'].unique()
tickers = stocks.keys()
handles_tickers = dict(zip(handles, tickers))
print(handles_tickers.items())
# Should be ('elonmusk', 'TSLA'), ('levie', 'BOX'), ('jack', 'TWTR'), ('Benioff', 'CRM'), ('richardbranson', 'SPCE'), ('JohnLegere', 'TMUS')

dict_items([('elonmusk', 'TSLA'), ('levie', 'BOX'), ('jack', 'TWTR'), ('Benioff', 'CRM'), ('richardbranson', 'SPCE'), ('JohnLegere', 'TMUS')])


In [9]:
print(stocks[list(stocks.keys())[0]][['Date','Open', 'Close']].head(3))

        Date       Open      Close
0 2016-08-23  44.863998  44.967999
1 2016-08-24  45.410000  44.523998
2 2016-08-25  44.622002  44.192001


In [10]:
# combine open/closed prices
stocks_org = {}
for stock in stocks:
    new_stock = stocks[stock].reset_index()
    new_stock =  stocks[stock][['Date','Open', 'Close']]
    new_stock.columns = new_stock.columns.str.lower()
    stocks_org[stock] = organize_stocks(new_stock)
    stocks_org[stock]['shifted'] = stocks_org[stock]['price'].shift() # number of lags (default is 1)
    stocks_org[stock]['change'] = stocks_org[stock]['price'].div(stocks_org[stock]['shifted'])
    stocks_org[stock]['return'] = stocks_org[stock]['change'].sub(1).mul(100)
print(stocks_org[list(stocks.keys())[0]].head(6))

                 date      price    shifted    change    return
0 2016-08-23 09:30:00  44.863998        NaN       NaN       NaN
0 2016-08-23 16:00:00  44.967999  44.863998  1.002318  0.231812
1 2016-08-24 09:30:00  45.410000  44.967999  1.009829  0.982924
1 2016-08-24 16:00:00  44.523998  45.410000  0.980489 -1.951116
2 2016-08-25 09:30:00  44.622002  44.523998  1.002201  0.220114
2 2016-08-25 16:00:00  44.192001  44.622002  0.990363 -0.963651


In [11]:
# merge into single dataframe 
stocks_full = pd.DataFrame()
for stock in stocks_org:
    stocks_org[stock]['ticker'] = stock
    stocks_full = stocks_full.append(stocks_org[stock])
stocks_full['date'] = stocks_full['date'].dt.tz_localize(tz='US/Eastern')
stocks_full = stocks_full.set_index('date')

In [12]:
tweets_combined['date'] = pd.to_datetime(tweets_combined['date'])
tweets_combined['date'] = tweets_combined['date'].dt.tz_localize(tz='US/Eastern')

In [15]:
combined_df = combine_tweets_stocks(tweets_combined, stocks_full)
wrangled_df = fix_closed_market_days(combined_df.reset_index(drop=True))
wrangled_df.head()

,tweet,num_tweets,username,price,shifted,change,return,ticker
date,,,,,,,,
2016-08-23 16:00:00-04:00,"[Journalist Q&amp;A for 30 mins and embargo ends at 12:30 2016-08-23 15:02:10 EDT, Tesla product announcement at noon California time today 2016-08-23 11:23:03 EDT]",2,elonmusk,44.967999,44.863998,1.002318,0.231812,TSLA
2016-08-30 16:00:00-04:00,"[@Kotaku one of my favorite games as a kid 2016-08-27 21:24:53 EDT, @BelovedRevol Making progress. Maybe something to announce in a few months. Have played all prior Deus Ex. Not this one yet. 2016-08-27 20:46:04 EDT, Thanks for the longstanding faith in SpaceX. We very much look forward to doing this milestone flight with you. https://t.co/U2UFez0OhY 2016-08-30 10:29:42 EDT]",3,elonmusk,42.268002,43.222000,0.977928,-2.207206,TSLA
2016-08-31 16:00:00-04:00,"[@Lockyep Not allowed, according to HK regulations. Happy to do it if regs change. 2016-08-31 11:43:00 EDT, We need to do one more minor rev on 8.0 and then will go to wide release in a few weeks 2016-08-31 11:27:35 EDT, Writing post now with details. Will publish on Tesla website later today. 2016-08-31 11:25:12 EDT, Major improvements to Autopilot coming with V8.0 and 8.1 software (std OTA update) primarily through advanced processing of radar signals 2016-08-31 11:23:36 EDT, @newscientist uh oh 2016-08-31 11:20:27 EDT]",5,elonmusk,42.402000,42.085999,1.007508,0.750848,TSLA
2016-09-01 16:00:00-04:00,[Loss of Falcon vehicle today during propellant fill operation. Originated around upper stage oxygen tank. Cause still unknown. More soon. 2016-09-01 13:07:54 EDT],1,elonmusk,40.153999,41.801998,0.960576,-3.942392,TSLA
2016-09-02 09:30:00-04:00,"[Finishing Autopilot blog postponed to end of weekend 2016-09-01 16:34:05 EDT, @scrappydog yes. This seems instant from a human perspective, but it really a fast fire, not an explosion. Dragon would have been fine. 2016-09-01 18:48:38 EDT]",2,elonmusk,40.466000,40.153999,1.007770,0.777009,TSLA


In [14]:
pd.to_pickle(wrangled_df, "./data/wrangled_df")